# Setup

In [1]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try:
    #import google.colab # type: ignore
    #from google.colab import output
    %pip install sae-lens transformer-lens
except:
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.0/920.0 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import os

from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Using device: cuda


In [4]:
from torch import nn, Tensor
from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple

# change input params here

In [ ]:
model_name = "tiny-stories-1L-21M"

# load model

In [5]:
from transformer_lens import HookedTransformer

In [6]:
model = HookedTransformer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/269M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


# load sae using saelens

## load

To get feature actvs, use sae_lens class method. To do this, you must load the sae as the sae class (wrapper over torch model).

In [31]:
model_name = "tiny-stories-1L-21M"
layer_name = "blocks.0.hook_mlp_out"
hook_layer = 0
d_in = 1024
expa_fac = 32
wandb_project = "sae_" + model_name+"_MLP" + str(hook_layer) + "_df-" + str(d_in * expa_fac)
drive_save_path = '/content/drive/MyDrive/'+wandb_project+'.pth'

In [8]:
total_training_steps = 30_000  # probably we should do more
batch_size = 4096
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name=model_name,  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    hook_name=layer_name,  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_layer=hook_layer,  # Only one layer in the model.
    d_in=d_in,  # the width of the mlp output.
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    is_dataset_tokenized=True,
    streaming=True,  # we could pre-download the token dataset if it was small.
    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor= expa_fac,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    scale_sparsity_penalty_by_decoder_norm=True,
    decoder_heuristic_init=True,
    init_encoder_as_decoder_transpose=True,
    normalize_activations="expected_average_only_in",
    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size_tokens=batch_size,
    context_size=512,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size_prompts=16,
    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project=wandb_project,
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype="float32"
)

Run name: 32768-L1-5-LR-5e-05-Tokens-1.229e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 30000
Total wandb updates: 1000
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 30 times.
Number tokens in sparsity calculation window: 4.10e+06


In [10]:
# https://github.com/jbloomAus/SAELens/blob/main/sae_lens/sae.py#L13
from sae_lens import SAEConfig

cfg_dict = cfg.to_dict()
cfg_dict['finetuning_scaling_factor'] = 0

sae_cfg = SAEConfig.from_dict(cfg_dict)

from sae_lens import SAE
sae = SAE(sae_cfg)

state_dict = torch.load(drive_save_path)
sae.load_state_dict(state_dict)

<All keys matched successfully>

## save decoder weights

In [ ]:
weight_matrix_np = sae.W_dec.cpu()

In [ ]:
import pickle
with open('ts-1L-21M_Wdec.pkl', 'wb') as f:
    pickle.dump(weight_matrix_np, f)

In [ ]:
# from google.colab import files
# files.download('weight_matrix.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp ts-1L-21M_Wdec.pkl /content/drive/MyDrive/

# load dataset

Need load model tokenizer and sae params before obtain dataset

In [11]:
from datasets import load_dataset

## any samples

In [12]:
# sae.cfg.context_size

In [13]:
from transformer_lens.utils import tokenize_and_concatenate

dataset = load_dataset("roneneldan/TinyStories", streaming=False)
test_dataset = dataset['validation']

token_dataset = tokenize_and_concatenate(
    dataset = test_dataset,
    tokenizer = model.tokenizer, # type: ignore
    streaming=True,
    # max_length=sae.cfg.context_size,
    max_length=128,
    # add_bos_token=sae.cfg.prepend_bos,
    add_bos_token=False,
)

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

Map:   0%|          | 0/21990 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (10434 > 2048). Running this sequence through the model will result in indexing errors


In [14]:
# sae.cfg.prepend_bos

In [15]:
batch_tokens = token_dataset[:500]["tokens"]
batch_tokens.shape

torch.Size([500, 128])

### save data

In [16]:
import pickle

In [17]:
with open('batch_tokens_anySamps_v1.pkl', 'wb') as f:
    pickle.dump(batch_tokens, f)

In [18]:
!cp batch_tokens_anySamps_v1.pkl /content/drive/MyDrive/

In [19]:
# check if saved
file_path = '/content/drive/MyDrive/batch_tokens_anySamps_v1.pkl'
with open(file_path, 'rb') as f:
    batch_tokens = pickle.load(f)

# model 1- interpret features

## get LLM actvs

In [20]:
h_store = torch.zeros((batch_tokens.shape[0], batch_tokens.shape[1], model.cfg.d_model), device=model.cfg.device)
h_store.shape

torch.Size([500, 128, 1024])

In [21]:
def store_h_hook(
    pattern: Float[Tensor, "batch seqlen d_model"],
    hook
):
    h_store[:] = pattern  # this works b/c changes values, not replaces entire thing

In [22]:
model.run_with_hooks(
    batch_tokens,
    return_type = None,
    fwd_hooks=[
        (layer_name, store_h_hook),
    ]
)

## get SAE actvs

In [23]:
sae.eval()  # prevents error if we're expecting a dead neuron mask for who grads
with torch.no_grad():
    feature_acts = sae.encode(h_store)

In [36]:
feature_acts.shape

torch.Size([500, 128, 32768])

In [39]:
def count_nonzero_features(feature_acts):
    # Count the number of 0s in the tensor
    # num_zeros = (feature_acts == 0).sum().item() # OOM
    # chunk
    num_zeros = 0
    for i in range(feature_acts.size(0)):
        for j in range(feature_acts.size(1)):
            num_zeros += (feature_acts[i, j, :] == 0).sum().item()

    # Count the number of nonzeroes in the tensor
    # num_ones = (feature_acts > 0).sum().item()
    num_ones = 0
    for i in range(feature_acts.size(0)):
        for j in range(feature_acts.size(1)):
            num_ones += (feature_acts[i, j, :] > 0).sum().item()

    # Calculate the percentage of 1s over 0s
    if num_zeros > 0:
        perc_ones_over_total = (num_ones / (num_ones + num_zeros)) * 100
    else:
        perc_ones_over_total = float('inf')  # Handle division by zero
    return perc_ones_over_total
count_nonzero_features(feature_acts)

0.08303112983703613

## save actvs

Now you have to save actvs, bc saelens not compatible with umap OR cosine sim lib

In [32]:
import pickle

In [33]:
acts_save_path = 'fActs_ts_1L_21M_anySamps_' + "_df-" + str(d_in * expa_fac) + '_v1.pkl'
with open(acts_save_path, 'wb') as f:
    pickle.dump(feature_acts, f)

In [41]:
'fActs_ts_1L_21M_anySamps_' + "_df-" + str(d_in * expa_fac) + '_v1.pkl'

'fActs_ts_1L_21M_anySamps__df-32768_v1.pkl'

In [42]:
!cp fActs_ts_1L_21M_anySamps__df-32768_v1.pkl /content/drive/MyDrive/

In [43]:
# check if saved
file_path = '/content/drive/MyDrive/' + acts_save_path
with open(file_path, 'rb') as f:
    feature_acts = pickle.load(f)

## interpret by dataset examples, keyword search

In [44]:
from rich import print as rprint

In [26]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
) -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [45]:
def store_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        s += f'batchID: {batch_idx}, '
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'
        feat_samps.append(seq)
    return s, feat_samps

In [ ]:
# store feature : str of top 5 snippets (contains batchID, val, seq in each substr)
feat_snip_dict_strs = {}
feat_snip_dict_lst = {}
samp_m = 5

for feature_idx in range(feature_acts.shape[-1]):
    if feature_idx % 1000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts, feature_idx, samp_m, batch_tokens=batch_tokens)
    disp_str, feat_samps = store_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens)
    feat_snip_dict_strs[feature_idx] = disp_str
    feat_snip_dict_lst[feature_idx] = feat_samps

In [48]:
relevant_features = {}

# keyword = 'princess'
keyword = '[bold u dark_orange] princess[/]'
relevant_features[keyword] = []
for f_id, str_output in feat_snip_dict_strs.items():
    if keyword in str_output:
        relevant_features[keyword].append(f_id)
        print(f'Feature: {f_id}')
        rprint(str_output)

Feature: 915


batchID: 0, Act = 4.38, Seq = ", there lived a rhinoceros named R"
batchID: 176, Act = 0.69, Seq = " said, "I love avocados!" and"
batchID: 194, Act = 0.56, Seq = ", and a fancy teapot. They sat"
batchID: 130, Act = 0.19, Seq = " twirled around in her princess dress, she noticed"
batchID: 175, Act = 0.15, Seq = " grocery store to buy some avocados. They"

Feature: 5626


batchID: 227, Act = 1.07, Seq = " knew that Jack was a hero who had helped many"
batchID: 427, Act = 0.68, Seq = " "You're my best friend, Olive. I"
batchID: 132, Act = 0.65, Seq = " for being such a good helper, Lily." Lily"
batchID: 361, Act = 0.63, Seq = " You are a very nice princess."
batchID: 107, Act = 0.49, Seq = " for being such a good helper. She knew that"

Feature: 5964


...

batchID: 361, Act = 0.69, Seq = " You are a very nice princess."
batchID: 296, Act = 0.69, Seq = " End. He's my friend. Can he play"
batchID: 298, Act = 0.65, Seq = ". I'm Lily's friend. Can I stay"
batchID: 297, Act = 0.57, Seq = "an. He's my friend. Can he stay"
batchID: 353, Act = 0.45, Seq = ". I love you, Sara."\n\nS"

Feature: 30033


batchID: 70, Act = 1.58, Seq = " anymore.\n\nThe princess felt bad and wanted"
batchID: 71, Act = 0.89, Seq = " that day on, the princess learned that it's"
batchID: 71, Act = 0.77, Seq = "I forgive you." The princess was happy and they"
batchID: 70, Act = 0.62, Seq = " want to talk to the princess anymore.\n\n"
batchID: 327, Act = 0.45, Seq = " with the king and the queen?"\n\n""

Feature: 30658


batchID: 188, Act = 1.52, Seq = " You are such a great teacher." \n\n"
batchID: 132, Act = 0.93, Seq = " for being such a good helper, Lily." Lily"
batchID: 361, Act = 0.78, Seq = " You are a very nice princess."
batchID: 251, Act = 0.76, Seq = " Lily! You're so creative. I love your"
batchID: 107, Act = 0.67, Seq = " for being such a good helper. She knew that"

In [55]:
# store feature : lst of top strs
feat_snip_dict_lst = {}
samp_m = 5

for feature_idx in range(feature_acts.shape[-1]):
    if feature_idx % 2000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts, feature_idx, samp_m, batch_tokens=batch_tokens)
    feat_snip_dict_lst[feature_idx] = store_top_sequences_asLst(ds_top_acts_indices, ds_top_acts_values, batch_tokens)

Feature:  0
Feature:  2000
Feature:  4000
Feature:  6000
Feature:  8000
Feature:  10000
Feature:  12000
Feature:  14000
Feature:  16000
Feature:  18000
Feature:  20000
Feature:  22000
Feature:  24000
Feature:  26000
Feature:  28000
Feature:  30000
Feature:  32000


## save datasamps for features

In [53]:
from google.colab import files

In [54]:
import pickle
with open('feat_snip_dict_strs.pkl', 'wb') as f:
    pickle.dump(feat_snip_dict_strs, f)
files.download('feat_snip_dict_strs.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
import json

# Transform the dictionary into a list of dictionaries
feature_top_samps_lst = [{"feature": feature, "strings": strings} for feature, strings in feat_snip_dict_lst.items()]

# Save to a JSON file
output_file = 'feature_top_samps_lst.json'
with open(output_file, 'w') as f:
    json.dump(feature_top_samps_lst, f, indent=4)

files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# model 2- interpret features

## load model and sae

In [ ]:
model_2 = HookedTransformer.from_pretrained("tiny-stories-2L-33M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/323M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [ ]:
model_name = "tiny-stories-2L-33M"
layer_name = "blocks.0.hook_mlp_out"
hook_layer = 0
d_in = 1024
expa_fac = 32
wandb_project = "sae_" + model_name+"_MLP" + str(hook_layer) + "_df-" + str(d_in * expa_fac)
drive_save_path = '/content/drive/MyDrive/'+wandb_project+'.pth'

In [ ]:
total_training_steps = 30_000  # probably we should do more
batch_size = 4096
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name=model_name,  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    hook_name=layer_name,  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_layer=hook_layer,  # Only one layer in the model.
    d_in=d_in,  # the width of the mlp output.
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    is_dataset_tokenized=True,
    streaming=True,  # we could pre-download the token dataset if it was small.
    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor= expa_fac,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    scale_sparsity_penalty_by_decoder_norm=True,
    decoder_heuristic_init=True,
    init_encoder_as_decoder_transpose=True,
    normalize_activations="expected_average_only_in",
    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size_tokens=batch_size,
    context_size=512,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size_prompts=16,
    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project=wandb_project,
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype="float32"
)

Run name: 16384-L1-5-LR-5e-05-Tokens-1.229e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 30000
Total wandb updates: 1000
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 30 times.
Number tokens in sparsity calculation window: 4.10e+06


In [ ]:
# https://github.com/jbloomAus/SAELens/blob/main/sae_lens/sae.py#L13
from sae_lens import SAEConfig

cfg_dict = cfg.to_dict()
cfg_dict['finetuning_scaling_factor'] = 0

sae_cfg = SAEConfig.from_dict(cfg_dict)

from sae_lens import SAE
sae = SAE(sae_cfg)

state_dict = torch.load(drive_save_path)
sae.load_state_dict(state_dict)

## save decoder weights

In [ ]:
weight_matrix_np = sae_2.W_dec.cpu()

In [ ]:
import pickle
with open('ts-2L-33M_Wdec.pkl', 'wb') as f:
    pickle.dump(weight_matrix_np, f)

In [ ]:
# from google.colab import files
# files.download('weight_matrix.pkl')

In [ ]:
!cp ts-2L-33M_Wdec.pkl /content/drive/MyDrive/

## get LLM actvs

In [ ]:
h_store = torch.zeros((batch_tokens.shape[0], batch_tokens.shape[1], model.cfg.d_model), device=model.cfg.device)
h_store.shape

torch.Size([500, 128, 1024])

In [ ]:
def store_h_hook(
    pattern: Float[Tensor, "batch seqlen d_model"],
    hook
):
    h_store[:] = pattern  # this works b/c changes values, not replaces entire thing

In [ ]:
model_2.run_with_hooks(
    batch_tokens,
    return_type = None,
    fwd_hooks=[
        (layer_name, store_h_hook),
    ]
)

## get SAE actvs

In [ ]:
sae_2.eval()  # prevents error if we're expecting a dead neuron mask for who grads
with torch.no_grad():
    feature_acts_2 = sae_2.encode(h_store)

In [ ]:
def count_nonzero_features(feature_acts):
    # Count the number of 0s in the tensor
    num_zeros = (feature_acts == 0).sum().item()

    # Count the number of nonzeroes in the tensor
    num_ones = (feature_acts > 0).sum().item()

    # Calculate the percentage of 1s over 0s
    if num_zeros > 0:
        perc_ones_over_total = (num_ones / (num_ones + num_zeros)) * 100
    else:
        perc_ones_over_total = float('inf')  # Handle division by zero
    return perc_ones_over_total
count_nonzero_features(feature_acts_2)

0.21000337600708008

## save actvs

Now you have to save actvs, bc saelens not compatible with umap OR cosine sim lib

In [ ]:
with open('fActs_ts_2L_33M_anySamps_v1.pkl', 'wb') as f:
    pickle.dump(feature_acts_2, f)

In [ ]:
!cp fActs_ts_2L_33M_anySamps_v1.pkl /content/drive/MyDrive/

In [ ]:
# check if saved
file_path = '/content/drive/MyDrive/fActs_ts_2L_33M_anySamps_v1.pkl'
with open(file_path, 'rb') as f:
    feature_acts_2 = pickle.load(f)

## get top features

In [ ]:
# Get the top k largest activations for feature neurons, not batch seq. use , dim=-1
# if want to get highest batch, use dim=0
feat_k = 100
top_acts_values, top_acts_indices = feature_acts_2.topk(feat_k, dim=-1)

print(top_acts_indices.shape)
top_acts_values.shape

torch.Size([500, 128, 100])


torch.Size([500, 128, 100])

## interpret top features by dataset examples

In [ ]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
) -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [ ]:
from rich import print as rprint
def display_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # s += f'{batch_idx}\n'
        s += f'batchID: {batch_idx}, '
        # Get the sequence as a string (with some padding on either side of our sequence)
        seq_start = max(seq_idx - 5, 0)
        # seq_end = min(seq_idx + 5, all_tokens.shape[1])
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        # Loop over the sequence, adding each token to the string (highlighting the token with the large activations)
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        # Print the sequence, and the activation value
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'

    rprint(s)

In [ ]:
# get top samp_m tokens for all top feat_k feature neurons
samp_m = 5

for feature_idx in top_acts_indices[0, -1, :]:
    feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_2, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  12261


batchID: 30, Act = 5.21, Seq = " a big bag of oats. He believed that if"
batchID: 409, Act = 4.62, Seq = " they see a big hedge. The hedge is green"
batchID: 0, Act = 4.46, Seq = "oxy found an icy hill."
batchID: 23, Act = 3.92, Seq = " saw a big candy store. He was very happy"
batchID: 61, Act = 3.70, Seq = " something unknown in the mud. It was a shiny"

Feature:  14599


batchID: 168, Act = 4.94, Seq = " he saw a big barrel. It was so big"
batchID: 7, Act = 4.27, Seq = " sour lemon on the ground. He wanted to play"
batchID: 46, Act = 4.23, Seq = " small key on the ground."
batchID: 30, Act = 4.20, Seq = " a big bag of oats. He believed that if"
batchID: 219, Act = 4.14, Seq = " shiny rock on the ground. She picked it up"

Feature:  9841


...

batchID: 162, Act = 2.27, Seq = " but none of them fit him well. \n"
batchID: 163, Act = 1.90, Seq = " Bambi that fit him just right. B"
batchID: 154, Act = 1.84, Seq = " a hard shell that protected him from danger. One"
batchID: 45, Act = 1.61, Seq = " in the park had led them to help the little"
batchID: 198, Act = 1.34, Seq = " that his dad had given him a special key to"

Feature:  47


batchID: 0, Act = 0.00, Seq = "Spot. Spot saw the shiny"
batchID: 0, Act = 0.00, Seq = "Spot. Spot saw the"
batchID: 0, Act = 0.00, Seq = "Spot. Spot saw the shiny car"
batchID: 0, Act = 0.00, Seq = "Spot. Spot saw the shiny car and said"
batchID: 0, Act = 0.00, Seq = "Spot. Spot saw the shiny car and"

Feature:  48


batchID: 101, Act = 5.19, Seq = ". She loved going to the shop with her mom"
batchID: 143, Act = 3.59, Seq = " going on a trip to the park. Timmy"
batchID: 453, Act = 3.54, Seq = " Timmy loved going to the library with his mom"
batchID: 83, Act = 3.33, Seq = ". She loved going to the zoo with her mom"
batchID: 185, Act = 3.31, Seq = " day, she went to the park to play with"

## only get features with keyword

In [ ]:
from rich import print as rprint

In [ ]:
def store_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        s += f'batchID: {batch_idx}, '
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'
    return s

In [ ]:
# store feature : str of top 5 snippets (contains batchID, val, seq in each substr)
feat_snip_dict_2 = {}
samp_m = 5

for feature_idx in range(feature_acts_2.shape[-1]):
    if feature_idx % 500 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_2, feature_idx, samp_m, batch_tokens=batch_tokens)
    feat_snip_dict_2[feature_idx] = store_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens)

Feature:  0
Feature:  500
Feature:  1000
Feature:  1500
Feature:  2000
Feature:  2500
Feature:  3000
Feature:  3500
Feature:  4000
Feature:  4500
Feature:  5000
Feature:  5500
Feature:  6000
Feature:  6500
Feature:  7000
Feature:  7500
Feature:  8000
Feature:  8500
Feature:  9000
Feature:  9500
Feature:  10000
Feature:  10500
Feature:  11000
Feature:  11500
Feature:  12000
Feature:  12500
Feature:  13000
Feature:  13500
Feature:  14000
Feature:  14500
Feature:  15000
Feature:  15500
Feature:  16000


In [ ]:
relevant_features = {}
# keyword = 'princess'
keyword = '[bold u dark_orange] princess[/]'
relevant_features[keyword] = []
for f_id, str_output in feat_snip_dict_2.items():
    if keyword in str_output:
        relevant_features[keyword].append(f_id)
        print(f'Feature: {f_id}')
        rprint(str_output)

Feature: 3101


batchID: 29, Act = 1.53, Seq = " smell.\n\nThe whale swam back to"
batchID: 29, Act = 1.43, Seq = " in green plants. The whale rubbed against the rock"
batchID: 29, Act = 1.30, Seq = " in the sea. The whale learned that it is"
batchID: 71, Act = 0.98, Seq = " that day on, the princess learned that it's"
batchID: 490, Act = 0.97, Seq = " and flew away. The poppy was not hurt,"

Feature: 3635


batchID: 327, Act = 2.90, Seq = "Wow, Tom, your dragon is cool," Lily"
batchID: 130, Act = 2.37, Seq = " twirled around in her princess dress, she noticed"
batchID: 331, Act = 2.34, Seq = " castle with you and your dragon."\n\nThey"
batchID: 111, Act = 2.07, Seq = " He said goodbye to his snowman and promised to"
batchID: 382, Act = 1.95, Seq = " to their room with their butter. They find some"

Feature: 5430


...

batchID: 361, Act = 3.55, Seq = " You are a very nice princess."
batchID: 188, Act = 2.71, Seq = " You are such a great teacher." \n\n"
batchID: 150, Act = 2.26, Seq = " she wanted to be an actor when she grew up"
batchID: 405, Act = 2.23, Seq = ", "I am a doctor. I need the"
batchID: 227, Act = 2.21, Seq = " knew that Jack was a hero who had helped many"

Feature: 10868


batchID: 129, Act = 6.21, Seq = " and pretend to be a princess. One day,"
batchID: 359, Act = 4.85, Seq = " pretend that Anna is a princess who lives in a"
batchID: 104, Act = 3.56, Seq = ". He felt like a king with his cushion throne"
batchID: 150, Act = 3.06, Seq = " stage pretended to be different characters. They would act"
batchID: 150, Act = 2.68, Seq = " she wanted to be an actor when she grew up"

Feature: 16096


batchID: 287, Act = 4.98, Seq = " Ben liked to play pirates in the garden."
batchID: 291, Act = 4.11, Seq = ". They decided to play pirates another day, when"
batchID: 12, Act = 3.41, Seq = ". They pretended to be kings and queens. They"
batchID: 359, Act = 3.06, Seq = " pretend that Anna is a princess who lives in a"
batchID: 129, Act = 2.65, Seq = " and pretend to be a princess. One day,"

In [ ]:
len(relevant_features[keyword])

10

## save ds for features

In [ ]:
def store_top_sequences_asLst(top_acts_indices, top_acts_values, batch_tokens):
    # s = ""
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # s += f'batchID: {batch_idx}, '
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        # s += f'Act = {value:.2f}, Seq = "{seq}"\n'
        feat_samps.append(seq)
    return feat_samps

In [ ]:
# store feature : lst of top strs
feat_snip_dict = {}
samp_m = 5

for feature_idx in range(feature_acts_2.shape[-1]):
    if feature_idx % 500 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_2, feature_idx, samp_m, batch_tokens=batch_tokens)
    feat_snip_dict[feature_idx] = store_top_sequences_asLst(ds_top_acts_indices, ds_top_acts_values, batch_tokens)

Feature:  0
Feature:  500
Feature:  1000
Feature:  1500
Feature:  2000
Feature:  2500
Feature:  3000
Feature:  3500
Feature:  4000
Feature:  4500
Feature:  5000
Feature:  5500
Feature:  6000
Feature:  6500
Feature:  7000
Feature:  7500
Feature:  8000
Feature:  8500
Feature:  9000
Feature:  9500
Feature:  10000
Feature:  10500
Feature:  11000
Feature:  11500
Feature:  12000
Feature:  12500
Feature:  13000
Feature:  13500
Feature:  14000
Feature:  14500
Feature:  15000
Feature:  15500
Feature:  16000


In [ ]:
import json

# Transform the dictionary into a list of dictionaries
feature_top_samps_lst = [{"feature": feature, "strings": strings} for feature, strings in feat_snip_dict.items()]

# Save to a JSON file
output_file = 'feature_top_samps_lst_2L_MLP0.json'
with open(output_file, 'w') as f:
    json.dump(feature_top_samps_lst, f, indent=4)

print(f"Data saved to {output_file}")

Data saved to feature_top_samps_lst_2L_MLP0.json


In [ ]:
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# store feature : str of top 5 snippets (contains batchID, val, seq in each substr)
feat_snip_dict_strs = {}
samp_m = 5

for feature_idx in range(feature_acts_2.shape[-1]):
    if feature_idx % 500 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_2, feature_idx, samp_m, batch_tokens=batch_tokens)
    feat_snip_dict_strs[feature_idx] = store_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens)

Feature:  0
Feature:  500
Feature:  1000
Feature:  1500
Feature:  2000
Feature:  2500
Feature:  3000
Feature:  3500
Feature:  4000
Feature:  4500
Feature:  5000
Feature:  5500
Feature:  6000
Feature:  6500
Feature:  7000
Feature:  7500
Feature:  8000
Feature:  8500
Feature:  9000
Feature:  9500
Feature:  10000
Feature:  10500
Feature:  11000
Feature:  11500
Feature:  12000
Feature:  12500
Feature:  13000
Feature:  13500
Feature:  14000
Feature:  14500
Feature:  15000
Feature:  15500
Feature:  16000


In [ ]:
import pickle
with open('feat_snip_dict_strs_2L_MLP0.pkl', 'wb') as f:
    pickle.dump(feat_snip_dict_strs, f)
files.download('feat_snip_dict_strs_2L_MLP0.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>